# Project Social Web - Conspiracy theories

Explaining the project...

## 1. Install and import packages

In [1]:
!pip3 install -U textblob

Requirement already up-to-date: textblob in /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages (0.15.3)
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import tweepy as tw
import nltk
from nltk import bigrams
from nltk.corpus import stopwords
import re
import networkx as nx
from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Yossigijrath/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 2. Twitter setup

In [3]:
consumer_key = 'N4J4jiQuGW7DZKLvUXg1DUYlw'
consumer_secret = 'HwhmSr8rGqbzG9NlfaOSHl4No2RijgtJwtDCQEFNZ5vlSPzYbj'
access_token = '158063657-YJjxwBnlFS6GGn0jkhnqA6PIjzVRC9UhACSjCWrX'
access_token_secret = 'kuwpm8QnoNIX3bTkgOrlhjbdEKYxRiFqxqRXXRdLGeoJF'
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [4]:
# Define the search term and the date_since date as variables
hashtag_word = "#electionfraud"
search_words = "#electionfraud -filter:retweets"
collection_words = ['electionfraud']
date_since = "2020-07-01"

## 3. Twitter hashtag scrapping

In [ ]:
# Collect tweets
tweets = tw.Cursor(api.search,
                   q=search_words,
                   lang="en",
                   since=date_since).items(1000)

all_tweets = [tweet.text for tweet in tweets]

all_tweets[:5]

## 4. Pre-prossesing and cleaning

In [ ]:
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [ ]:
# Remove URLs
all_tweets_no_urls = [remove_url(tweet) for tweet in all_tweets]

# Create a sublist of lower case words for each tweet
words_in_tweet = [tweet.lower().split() for tweet in all_tweets_no_urls]

# Adding stopwords
stop_words = set(stopwords.words('english'))

# Remove stop words from each tweet list of words
tweets_nsw = [[word for word in tweet_words if not word in stop_words]
              for tweet_words in words_in_tweet]

# Remove collection words
tweets_nsw_nc = [[w for w in word if not w in collection_words]
                 for word in tweets_nsw]

all_words_nsw_nc = list(itertools.chain(*tweets_nsw_nc))

# Create counter of words in clean tweets
counts_nsw_nc = collections.Counter(all_words_nsw_nc)

## 5. Graph of most used words for given hashtag

In [ ]:
clean_tweets_ncw = pd.DataFrame(counts_nsw_nc.most_common(20),
                             columns=['words', 'count'])

fig, ax = plt.subplots(figsize=(8, 8))

# Plot horizontal bar graph
clean_tweets_ncw.sort_values(by='count').plot.barh(x='words',
                      y='count',
                      ax=ax,
                      color="red")

ax.set_title("Common Words Found in Tweets on %s" % hashtag_word)

plt.show()

## 6. Word bigrams and Network analysis

In [ ]:
terms_bigram = [list(bigrams(tweet)) for tweet in tweets_nsw_nc]

# Flatten list of bigrams in clean tweets
bigrams = list(itertools.chain(*terms_bigram))

# Create counter of words in clean bigrams
bigram_counts = collections.Counter(bigrams)

bigram_counts.most_common(20)
bigram_df = pd.DataFrame(bigram_counts.most_common(20),
                             columns=['bigram', 'count'])

bigram_df

In [ ]:
# Create dictionary of bigrams and their counts
d = bigram_df.set_index('bigram').T.to_dict('records')

# Create network plot 
G = nx.Graph()

# Create connections between nodes
for k, v in d[0].items():
    G.add_edge(k[0], k[1], weight=(v * 10))

G.add_node("conspiracy", weight=100)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

pos = nx.spring_layout(G, k=2)

# Plot networks
nx.draw_networkx(G, pos,
                 font_size=16,
                 width=3,
                 edge_color='grey',
                 node_color='red',
                 with_labels = False,
                 ax=ax)

# Create offset labels
for key, value in pos.items():
    x, y = value[0]+.135, value[1]+.045
    ax.text(x, y,
            s=key,
            bbox=dict(facecolor='blue', alpha=0.25),
            horizontalalignment='center', fontsize=13)
    
plt.show()

## 7. Sentiment analysis

In [ ]:
# Create textblob objects of the tweets
sentiment_objects = [TextBlob(tweet) for tweet in all_tweets_no_urls]

# Create list of polarity valuesx and tweet text
sentiment_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]

# Create dataframe containing the polarity value and tweet text
sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "tweet"])

# Remove polarity values equal to zero
sentiment_df = sentiment_df[sentiment_df.polarity != 0]

sentiment_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

# Plot histogram with break at zero
sentiment_df.hist(bins=[-1, -0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75, 1],
             ax=ax,
             color="purple")

plt.title("Sentiments from Tweets on %s" % hashtag_word)
plt.show()